In [4]:
import torch
# from mnist import *
import torch.nn as nn
import cv2
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision import datasets, transforms
import numpy as np
import torchvision
from skimage import io,transform

In [5]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [46]:
input_size = 784
hidden_size = 500
num_classes = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = torch.load('./data/MNIST_model.pt') #加载模型
# model = model.to(device)
# model.eval()    #把模型转为test模式

model = NeuralNet(input_size, hidden_size, num_classes).to(device)
model.load_state_dict(torch.load('./data/MNIST_model.pt'))
model.eval()

# print(model)

img = cv2.imread('./data/five.png')
# img = cv2.resize(img, (28, 28))

# following line may or may not be necessary
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
print(img)

# you can divide numpy arrays by a constant natively
img /= 255.0

normalized_8 = [(255 - x) * 1.0 / 255.0 for x in img ]
tensor_8 = torch.FloatTensor( normalized_8 )

# This makes a 4d tensor (batched image) with shape [1, channels, width, height]
img = torch.Tensor(tensor_8).unsqueeze(axis=0)



# height,width=img.shape
# dst=np.zeros((height,width),np.uint8)
# for i in range(height):
#     for j in range(width):
#         dst[i,j]=255-img[i,j]

# img = dst

# img=np.array(img).astype(np.float32)
# img=np.expand_dims(img,0)
# img=np.expand_dims(img,0)   #扩展后，为[1，1，28，28]
# img=torch.from_numpy(img)
# img = img.reshape(-1, 28*28).to(device)
#
# print(img.size())

# output = model(Variable(img))
output = model(img)
print(f'output: {output}')

prob = F.softmax(output, dim=1)
prob = Variable(prob)
print(f'prob: {prob}')

prob = prob.cpu().numpy()  #用GPU的数据训练的模型保存的参数都是gpu形式的，要显示则先要转回cpu，再转回numpy模式
print(prob)  #prob是10个分类的概率
pred = np.argmax(prob) #选出概率最大的一个
print(pred.item())

[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]


UFuncTypeError: Cannot cast ufunc 'divide' output from dtype('float64') to dtype('uint8') with casting rule 'same_kind'